In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import os
import time
from PIL import Image
import scipy.io as sio

# 超参数
EPOCH = 5
BATCH_SIZE = 64
LR = 0.001
DOWNLOAD_MNIST = False


In [2]:
data = sio.loadmat('cars_annos.mat')
class_names = data['class_names']
f_class = open('./label_map.txt','w')
 
num = 1
for j in range(class_names.shape[1]):
    class_name = str(class_names[0,j][0]).replace(' ','_')
#     print(num,class_name)
    f_class.write( str(num) + ' ' + class_name + '\n')
    num = num + 1
f_class.close()

In [32]:
data = sio.loadmat('cars_annos.mat')
annotations = data['annotations']
f_train = open('./mat2txt.txt','w')

num = 1
for i in range(annotations.shape[1]):
    name = str(annotations[0,i][0])[2:-2]
    test  = int(annotations[0,i][6])
    clas = int(annotations[0,i][5])

    name = str(name)
    clas = str(clas)
    test = str(test)
    f_train.write(str(num) + ' ' + name + ' ' + clas + ' ' + test+'\n')
    num = num + 1


f_train.close()

In [3]:
# 老数据集用这个

data = sio.loadmat('cars_train_annos.mat')
annotations = data['annotations']
f_train = open('./train.txt','w')
 
for i in range(annotations.shape[1]):
    num = int(annotations[0,i][4])
    num = str(num)
#     print(i,num)
    f_train.write(num+'\n')


f_train.close()

In [4]:
# PIL 读取 image
def default_loader(path='./data/cars_train'):
    try:
        img = Image.open(path)
        return img.convert('RGB')
    except:
        print(f'Cannot read image: {path}')

In [5]:
class customData(Dataset):
    
    # 将图片路径存入self.img_name,将对应的标签写入self.img_label里面
    def __init__(self, img_path, txt_path, dataset='', data_transforms=None, loader=default_loader):
        with open(txt_path) as input_file:
            lines = input_file.readlines()
            self.img_label = [int(line.strip()) for line in lines]
        
        self.img_name = []
        for root, dirs, files in os.walk(img_path):
            for name in sorted(files):
                self.img_name.append(os.path.join(img_path, name))
        self.data_transforms = data_transforms
        self.dataset = dataset
        self.loader = loader
        
    def __len__(self):
        return len(self.img_name)
    
    def __getitem__(self, item):
        img_name = self.img_name[item]
        label = self.img_label[item]
        img = self.loader(img_name)
        
        if self.data_transforms is not None:
            try:
                img = self.data_transforms[self.dataset](img)
            except:
                print(f'Cannot transform images: {image_name}')
                
        return img, label

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs, use_gpu):
    since = time.time()
    best_model_wts = model.state_dict()
    best_acc = 0.0
 
    for epoch in range(num_epochs):
        begin_time = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
 
        # Each epoch has a training and validation phase
        # for phase in ['train','val']:
        for phase in ['train']:
            count_batch = 0
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode
 
            running_loss = 0.0
            running_corrects = 0.0
 
            # Iterate over data.
            for data in dataloders[phase]:
                count_batch += 1
                # get the inputs
                inputs, labels = data
 
                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
 
                # zero the parameter gradients
                optimizer.zero_grad()
 
                # forward
                outputs = model(inputs)
                # print(outputs.data)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)
 
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                # statistics
                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data).to(torch.float32)
 
                # print result every 10 batch
                if count_batch%10 == 0:
                    batch_loss = running_loss / (batch_size*count_batch)
                    batch_acc = running_corrects / (batch_size*count_batch)
                    print('{} Epoch [{}] Batch [{}] Loss: {:.4f} Acc: {:.4f} Time: {:.4f}s'. \
                          format(phase, epoch, count_batch, batch_loss, batch_acc, time.time()-begin_time))
                    begin_time = time.time()
 
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
 
            # save model
            if phase == 'train':
                if not os.path.exists('output'):
                    os.makedirs('output')
                torch.save(model, 'output/resnet_epoch{}.pkl'.format(epoch))
 
            # deep copy the model
            # if phase == 'val' and epoch_acc > best_acc:
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
 
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    # print('Best val Acc: {:4f}'.format(best_acc))
    print('Best acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
if __name__ == '__main__':
 
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
       # 'val': transforms.Compose([
           # transforms.Resize(256),
           # transforms.CenterCrop(224),
           # transforms.ToTensor(),
           # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
       # ]),
    }
 
    # use_gpu = torch.cuda.is_available()   # 用gpu跑的话，可以把注释去掉
    # print(use_gpu)
 
    batch_size = 32
    num_class = 197  # 用196会报错
 
    # image_datasets = {x: customData(img_path='/ImagePath',
    #                                 txt_path=('/TxtFile/' + x + '.txt'),
    #                                 data_transforms=data_transforms,
    #                                 dataset=x) for x in ['train', 'val']}
 
    image_datasets = {x: customData(img_path=r'./data/cars_train/',
                                    txt_path='./'+ x + '.txt',
                                    data_transforms=data_transforms,
                                    dataset=x) for x in ['train']}  # ['train', 'val']
 
    # wrap your data and label into Tensor
    dataloders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train']} # ['train', 'val']
 
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train']}  # ['train', 'val']
 
    # get model and replace the original fc layer with your fc layer
    model_ft = models.resnet50(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    # print(num_ftrs)
    model_ft.fc = nn.Linear(num_ftrs, num_class)   # 将resnet最后全连接层的输出换成数据集的类别总数
 
    # if use gpu
    # if use_gpu:
    #     model_ft = model_ft.cuda()
 
    # define cost function
    criterion = nn.CrossEntropyLoss()
 
    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.005, momentum=0.9)
 
    # Decay LR by a factor of 0.2 every 5 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.2)
 
    # multi-GPU
    # model_ft = torch.nn.DataParallel(model_ft, device_ids=[0,1])
 
    # train model
    model_ft = train_model(model=model_ft,
                           criterion=criterion,
                           optimizer=optimizer_ft,
                           scheduler=exp_lr_scheduler,
                           num_epochs=EPOCH,
                           use_gpu=False)
 
 
 
 
    # save best model
    torch.save(model_ft,"output/best_resnet.pkl")

Epoch 0/4
----------


C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Epoch [0] Batch [10] Loss: 0.1670 Acc: 0.0063 Time: 149.0148s
train Epoch [0] Batch [20] Loss: 0.1675 Acc: 0.0031 Time: 133.4940s
train Epoch [0] Batch [30] Loss: 0.1673 Acc: 0.0073 Time: 132.8196s
train Epoch [0] Batch [40] Loss: 0.1673 Acc: 0.0078 Time: 124.6034s
train Epoch [0] Batch [50] Loss: 0.1669 Acc: 0.0100 Time: 127.5398s
train Epoch [0] Batch [60] Loss: 0.1657 Acc: 0.0109 Time: 123.1182s
train Epoch [0] Batch [70] Loss: 0.1646 Acc: 0.0129 Time: 123.6780s
train Epoch [0] Batch [80] Loss: 0.1635 Acc: 0.0176 Time: 122.2542s
train Epoch [0] Batch [90] Loss: 0.1624 Acc: 0.0201 Time: 122.9324s
train Epoch [0] Batch [100] Loss: 0.1612 Acc: 0.0225 Time: 121.8843s
train Epoch [0] Batch [110] Loss: 0.1600 Acc: 0.0256 Time: 122.2273s
train Epoch [0] Batch [120] Loss: 0.1584 Acc: 0.0312 Time: 121.7766s
train Epoch [0] Batch [130] Loss: 0.1571 Acc: 0.0346 Time: 121.5857s
train Epoch [0] Batch [140] Loss: 0.1558 Acc: 0.0362 Time: 121.6656s
train Epoch [0] Batch [150] Loss: 0.1543 Ac

C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\ProgramDat


----------
train Epoch [1] Batch [10] Loss: 0.1039 Acc: 0.2219 Time: 121.3327s
train Epoch [1] Batch [20] Loss: 0.1051 Acc: 0.2016 Time: 121.5767s
train Epoch [1] Batch [30] Loss: 0.1019 Acc: 0.2490 Time: 121.8542s
train Epoch [1] Batch [40] Loss: 0.1003 Acc: 0.2516 Time: 121.3917s
train Epoch [1] Batch [50] Loss: 0.0996 Acc: 0.2569 Time: 121.4477s
train Epoch [1] Batch [60] Loss: 0.0984 Acc: 0.2625 Time: 122.2531s
train Epoch [1] Batch [70] Loss: 0.0974 Acc: 0.2696 Time: 121.8416s
train Epoch [1] Batch [80] Loss: 0.0970 Acc: 0.2727 Time: 121.0778s
train Epoch [1] Batch [90] Loss: 0.0959 Acc: 0.2753 Time: 121.2371s
train Epoch [1] Batch [100] Loss: 0.0956 Acc: 0.2759 Time: 120.9286s
train Epoch [1] Batch [110] Loss: 0.0949 Acc: 0.2801 Time: 121.9893s
train Epoch [1] Batch [120] Loss: 0.0944 Acc: 0.2826 Time: 120.9121s
train Epoch [1] Batch [130] Loss: 0.0935 Acc: 0.2849 Time: 122.2959s
train Epoch [1] Batch [140] Loss: 0.0930 Acc: 0.2873 Time: 122.2713s
train Epoch [1] Batch [150] Los